In [1]:
import numpy as np
import pathlib
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

import matplotlib.pyplot as plt

In [2]:
input_path = os.path.join(os.path.join(pathlib.Path(globals()['_dh'][0]).parent, "data"), "js-fakes-16thSeparated.npz")
jsf = np.load(input_path, allow_pickle=True, encoding='latin1')

In [3]:
def add_token(input_stream, token, begin):
    token_shape = list(input_stream.shape)
    token_shape[0] = 1
    tens_tokens = torch.ones(token_shape) * token
    if begin == True:
        return torch.cat((tens_tokens, input_stream), axis = 0)
    else:
        return torch.cat((input_stream, tens_tokens), axis = 0) 

#[L, bs, F]

def song_transform(song, max_lenght):
    song = F.pad(song, 'constant', 0)
    song = add_token(song, 1, begin=True)
    song = song[:max_lenght]
    song = add_token(song, -1, begin=False)
    return song

In [11]:
#Create iterator that extract batch of variable size from jsf
max_song_length = np.asarray([len(song) for song in jsf['pitches']]).max()

class SongIterator():
    def __init__(self, song_list):
        self.internal_song_list = song_list

    def __iter__(self):
        #Capire cosa deve tornare
        return self
    
    def __next__(self):
        x = 1 #Capire cosa deve tornare
        return x 



520

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, num_heads_1, num_heads_2, bs, emb_notes, emb_f):
        super(TransformerBlock, self).__init__() # Seq, batch, features
        self.ln1 = nn.LayerNorm([bs * num_heads_2 * emb_notes, num_heads_1 * emb_f]) #layer norm: [L, bs, Emb_f, Emb_notes] -> [L, bs, Emb_f, Emb_notes]
        self.mha_f = nn.MultiheadAttention(num_heads_1 * emb_f, num_heads_1, dropout=0.25) # multi-head attention per features: [L, BS * Emb_notes, Emb_f] -> [L, BS * Emb_notes, Emb_f]
        self.ln2 = nn.LayerNorm([bs * num_heads_1 * emb_f, num_heads_2 * emb_notes]) #layer norm: [BS, L, Emb_f, Emb_notes] -> [BS, L, Emb_f, Emb_notes]
        self.mha_l = nn.MultiheadAttention(num_heads_2 * emb_notes, num_heads_2, dropout=0.25) # multi-head attention per lunghezza: [BS * Emb_f, L, Emb_notes] -> [BS * Emb_f, L, Emb_notes]
        self.ln3 = nn.LayerNorm([bs, num_heads_1 * emb_f, num_heads_2 * emb_notes]) #layer norm
        
        self.mlp = nn.Sequential(
            nn.Linear(num_heads_2 * emb_notes, num_heads_2 * emb_notes),  # Linear transformation
            nn.LayerNorm([bs, num_heads_1 * emb_f, num_heads_2 * emb_notes]),  # Layer normalization
            nn.ELU(),  # Activation function (ELU)
            nn.Linear(num_heads_2 * emb_notes, num_heads_2 * emb_notes)  # Linear transformation
        )

    
    def forward(self, x): # add various reshape
        #[L, bs, Emb_f, Emb_notes]
        print("Step 1, x shape = {}".format(x.shape))
        x_1 = x.transpose(2, 3).reshape((x.shape[0], x.shape[1] * x.shape[3], x.shape[2])) 
        #[L, BS * Emb_notes, Emb_f]
        print("Step 2, x_1 shape = {}".format(x_1.shape))
        norm_x_1 = self.ln1(x_1) 
        print("Step 3, x_1 shape = {}".format(x_1.shape))
        attn_output_1 = self.mha_f(norm_x_1, norm_x_1, norm_x_1)[0] # [0] selects the attention output, to be decided if padding is needed
        print("Step 4, attn_output_1 shape = {}".format(attn_output_1.shape))
        x_2 = x_1 + attn_output_1 # residual connection
        print("Step 4, x_2 shape = {}".format(x_2.shape))
        x_2 = x_2.reshape(x.shape[0], x.shape[1], x.shape[3], x.shape[2]).transpose(2,3).reshape(x.shape[0], x.shape[1] * x.shape[2], x.shape[3])
        #[L, BS * Emb_f, Emb_notes]
        print("Step 5, x_2 shape = {}".format(x_2.shape))
        norm_x_2 = self.ln2(x_2)
        print("Step 6, x_norm_x_2 shape = {}".format(norm_x_2.shape))
        attn_output_2 = self.mha_l(norm_x_2, norm_x_2, norm_x_2)[0] # [0] selects the attention output, to be decided if padding is needed
        print("Step 7, attn_output_2 shape = {}".format(attn_output_2.shape))
        x = x + attn_output_2.reshape(x.shape)
        print("Step 8, x shape = {}".format(x.shape))
        x = self.ln3(x)
        print("Step 9, x shape = {}".format(x.shape))
        x = self.mlp(x)
        print("Step 9, x shape = {}".format(x.shape))
        return x

In [ ]:
bs = 4
emb_notes = 32 
emb_f = 4
L = 150
num_heads_1 = 4
num_heads_2 = 4
trans_block = TransformerBlock(num_heads_1 = num_heads_1, num_heads_2 = num_heads_2, bs = bs, emb_notes = emb_notes, emb_f = emb_f)
#[L, bs, Emb_f, Emb_notes]

rand_input = torch.rand((L, bs, num_heads_1 * emb_f, num_heads_2 * emb_notes))


In [ ]:
rand_song = torch.randint(36,82,(L, bs, 4))
tokens = [0, 1, -1, -2] + list(range(36,82)) # 0: padding, 1: start of signal, -1: end of signal, -2: unknown

In [ ]:
#Discriminator might relize that padding = real example, find out how to avoid this -> Thus add an attenction mask for padding

#Discriminator -> Some Transformer block followed by taking the first token + linear + tanh

# Create a tranform operation that remove extra symbol, add 


In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
    
    def forward(self, x):
        return x
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
    
    def forward(self, x):
        return x